# 1. Load necessary packages.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster # for clustering the markers

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2. Load all datasets.

In [ ]:
df1 = pd.read_csv('../input/atlanta-crime-data-2020/COBRA-2020-OldRMS-09292020.csv')
df2 = pd.read_csv('../input/atlanta-crime-data-2020/COBRA-2020.csv')

# **3. Check each dataset.**

In [ ]:
df1

In [ ]:
df2

# **4. Drop unnecessary columns and concatenate datasets.**

In [ ]:
df1 = df1.drop(columns = ['apartment_office_prefix','Apartment_number','watch','location_type','UCR_Number'])

In [ ]:
df2 = df2.drop(columns = ['ibr_code'])

In [ ]:
df = pd.concat([df1,df2]).reset_index(drop = True)

In [ ]:
df

# **5. Save the new dataset to a CSV file.**

In [ ]:
df.to_csv('df.csv', index=False)

# 6. Preprocess data

**6.1 Add necessary columns.**

In [ ]:
keys = df.columns
df = df.dropna(subset = keys).reset_index(drop = True)

In [ ]:
df['rpt_month'] = df['rpt_date'].str.split('/').str[0]
df['rpt_day'] = df['rpt_date'].str.split('/').str[1]
df['rpt_year'] = df['rpt_date'].str.split('/').str[2]

df['occur_month'] = df['occur_date'].str.split('/').str[0]
df['occur_day'] = df['occur_date'].str.split('/').str[1]
df['occur_year'] = df['occur_date'].str.split('/').str[2]
df['occur_hour'] = df['occur_time'].str.split(':').str[0]

**6.2 Deal with datatype.**

In [ ]:
df = df.astype({'occur_year' : 'int32', 'occur_month' : 'int32', 'occur_day' : 'int32', 'occur_hour' : 'int32', 'rpt_year' : 'int32', 'rpt_month' : 'int32', 'rpt_day' : 'int32'})

**6.3 Dive into each column and drop wrong rows.**

In [ ]:
df['occur_year'].value_counts()

In [ ]:
df = df[(df['occur_year'] == 2019) | (df['occur_year'] == 2020)].reset_index(drop=True)

In [ ]:
df

In [ ]:
df['occur_month'].nunique()

In [ ]:
df['occur_day'].nunique()

In [ ]:
df['occur_hour'].nunique()

In [ ]:
df['rpt_year'].nunique()

In [ ]:
df['rpt_month'].nunique()

In [ ]:
df['rpt_day'].nunique()

In [ ]:
df['location'].value_counts().head(15)

In [ ]:
df['neighborhood'].value_counts()

# 7. Plot

In [ ]:
r_hours = df.groupby('occur_hour').count()
r_hours

In [ ]:
rh = r_hours['offense_id'].reset_index()

In [ ]:
rh

In [ ]:
plt.plot(rh['occur_hour'], rh['offense_id'],'b-.')
plt.title('# of Crime Occurred Over Hours')
plt.xlabel('Hours')
plt.ylabel('# of Crime')
plt.xticks(rh['occur_hour'], size = 11)
plt.yticks(size = 11)
plt.show()

In [ ]:
r_months = df.groupby('occur_month').count()
r_months

In [ ]:
rm = r_months['offense_id'].reset_index()
rm

In [ ]:
plt.plot(rm['occur_month'], rm['offense_id'], 'b--')
plt.title('# of Crime Occurred Over Months')
plt.xlabel('Months')
plt.ylabel('# of Crime')
plt.xticks(rm['occur_month'], size = 11)
plt.yticks(size = 11)
plt.show()

In [ ]:
r_neighbors = df.groupby('neighborhood').count()
r_neighbors

In [ ]:
rn = r_neighbors['offense_id'].reset_index()
rn

In [ ]:
rn = rn.sort_values(by = ['offense_id'], ascending = False)

In [ ]:
rn = rn.head(15)
rn

In [ ]:
plt.bar(rn['neighborhood'], rn['offense_id'])
plt.title('# of Crime Occurred Among Neighborhoods(top15)')
plt.xlabel('Neighborhood')
plt.ylabel('# of Crime')
plt.xticks(size = 11, rotation = 'vertical')
plt.yticks(size = 11)
plt.show()

**Call for map widget.**

In [ ]:
map = folium.Map(location=[33.8982, -84.2833], default_zoom_start=12)

In [ ]:
# add a marker for every record in the filtered data, use a clustered view
marker_cluster = MarkerCluster().add_to(map) # create marker clusters
for i in range(df.shape[0]):
    location = [df['lat'][i],df['long'][i]]
    tooltip = "Neighborhood: {}<br> Click for more".format(df["neighborhood"][i])
    folium.Marker(location, 
                  popup="""<i>Crime Address: </i> <br> <b>{}</b> <br>""".format(df['location'][i]), 
                  tooltip=tooltip).add_to(marker_cluster)


In [ ]:
map.save('map.html')